In [ ]:
%pip install opentrons prefect

In [ ]:
# source: https://support.opentrons.com/s/article/Running-the-robot-using-Jupyter-Notebook

# import opentrons.execute
import opentrons.simulate  # if you want to simulate the protocol first

# This is where you establish the API version for executing a protocol
# protocol = opentrons.execute.get_protocol_api('2.11')
# to simulate the protocol, replace opentrons.execute with opentrons.simulate
# like so
protocol = opentrons.simulate.get_protocol_api("2.16")

# Must home before running
protocol.home()

# Labware
plate = protocol.load_labware("nest_96_wellplate_200ul_flat", "2")

tiprack_1 = protocol.load_labware("opentrons_96_tiprack_1000ul", location="1")

# Pipette
p1000 = protocol.load_instrument("p1000_single_gen2", "right", tip_racks=[tiprack_1])

# Protocol steps
p1000.transfer(100, plate["A1"], plate["A2"])

# Output steps in protocol
for line in protocol.commands():
    print(line)

In [ ]:
from prefect import flow
from prefect.deployments import run_deployment
import time


# Parent prefect flow that spawns sub flows.
@flow
async def parent_flow() -> None:
    for i in range(10):
        run_deployment(name="<Flow_name>/<deployment_name>")

In [ ]:
R = 10
Y = 50
B = 40
mix_well = "A1"

# move_sensor_to_measurement_position()
# collect_sensor_data()
# move_sensor_back()
# mix_color(R, Y, B)
# move_sensor_to_measurement_position()
# collect_sensor_data()
# move_sensor_back()


run_deployment(
    name="mix_color/mix-color",
    parameters={"R": R, "Y": Y, "B": B, "mix_well": mix_well},
)
run_deployment(name="collect_sensor_data/collect-sensor-data")
...

In [ ]:
!prefect deployment inspect 'mix_color/mix-color' # to look at OpenAPI schema